In [ ]:
#Autores: Daniel Castillo, Karla Salas
from os import listdir
from os.path import isfile, join
#Para ver las palabras
from collections import Counter
import matplotlib.pyplot as plt
# nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split #particiones
from nltk.corpus import stopwords #Listas de stopwords
from nltk.tokenize import word_tokenize, sent_tokenize #Tokens
import re #regex
from itertools import chain #bigramas
import numpy as np
from sklearn.decomposition import PCA
from operator import itemgetter

## Limpiamos y tokenizamos
(Tomamos el código de la práctica anterior)

Tokenizamos por palabra, ya que nos interesa medir la probabilidad de transitar de una palabra a otra y no por subpalabras, para intentar, más adelante, responder con una oración dada una entrada.

In [ ]:
def get_txt(path):
    """
    Regresa una lista con el contenido de todos los archivos de un directorio

    Args:
        path (str): ruta de la carpeta
    """
    text = []
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    
    for file in onlyfiles:
         with open(path+"/"+file, 'rb') as f:
            text.append(f.read().decode('utf-8', 'replace'))
    return text

# Guardamos cada película en un diccionario
# cada entrada del diccionario es una lista con las peliculas leídas
movies = {}
movies["Pride & Prejudice"] = get_txt("../corpus/Pride & Prejudice")
movies["Marvel"] = get_txt("../corpus/Marvel")
movies["Christopher Nolan"] = get_txt("../corpus/Christopher Nolan")

In [ ]:
# tokens que necesitan ser limpiados del corpus y que no
# se encuentran en la lista de stopwords
more = ["_","-","'ve", "'ll", "'t", "'s", "'re", "'", "'m", "'d", "n't", "oh", "hey", "yeah","okay", "mr.", "miss", "mrs."]
stopwords_list = stopwords.words('english') + more

def get_tokens_clean(text):
    '''
    Genera los tokens de una cadena y los limpia
    (quita símbolos raros y stopwords)
    
    Args:
        text (str): cadena
    '''
    tokens = word_tokenize(text)
    clean = []
    pattern = r'[^a-z0-9\s]'
    for w in tokens:
        #quita stopwords y convierte a minúsculas
        w = re.sub(pattern,'', w.lower())
        if w not in stopwords_list and w != '':
            if  w == "na": #Para juntar gon na, wan na, etc.
                clean[-1] += w
            else:
                clean.append(w)

    return clean

In [ ]:
def merge_movies(col):
    '''
    Mezcla el corpus de cada película en una entrada del
    diccionario movies en un solo corpus, 
    guarda los diálogos por línea tokenizados y limpios

    Args:
        col (dic): Diccionario con los diálogos
    '''
    corpus = []
    #Iteramos sobre las películas
    for movie in col:
        print(movie, len(movies[movie]))
        for text in movies[movie]:
            #Cada texto se guarda por oraciones (contexto)
            corpus += [get_tokens_clean(s) for s in sent_tokenize(text)]
    return corpus

all_movies = merge_movies(movies)

In [ ]:
#Particiones train, test (30%), val (10%)
train_aux, test = train_test_split(all_movies, test_size=0.3)
train, val = train_test_split(all_movies, test_size=0.1)
print('Número de cadenas train:',len(train))
print('Número de cadenas val:',len(val))
print('Número de cadenas test:',len(test))

No es necesario quitar las hápax (palabras que solo aparecen una vez en el corpus) la red propuesta por Bengio ya las toma en consideración.

## Vocabulario (indexado)
Obtenemos los pares de entrenamiento a partir de contextos, bigramas.

In [ ]:
def get_words_and_indexes(corpus):
    # indices para cada palabra
    words_to_index = list(Counter( chain(*[sentence for sentence in corpus]) ).keys())
    # Agregamos etiquetas de inicio y fin al diccionario
    words_to_index.append('<BOS>')
    words_to_index.append('<EOS>')

    words_to_index = dict(zip(words_to_index, range(0, len(words_to_index) - 1)))
    
    # A cada línea de los diálogos se le agrega la etiqueta BOS al inicio y EOS al final
    new_corpus = [[len(words_to_index) - 2] + [words_to_index[t] for t in text] + [len(words_to_index) - 1] for text in train]
    # Se crean los bigramas
    bigrams = list(chain(*[zip(cad,cad[1:]) for cad in new_corpus]))
    return words_to_index, bigrams, new_corpus

# Para el entrenamiento con el conjunto grande
word_to_index, bigrams, new_corpus = get_words_and_indexes(train)
# Para el entrenamiento con el conjunto pequeño
word_to_indexV, bigramsV, new_corpusV = get_words_and_indexes(train)
print(len(bigramsV))

# Red Neuronal

In [ ]:
class Bengio:
    def __init__(self, input_dim, output_dim, num_examples):
        np.random.seed(0)
        #El número de rasgos que representan cada vector
        self.input_dim = input_dim
        #El total de clases que arrojará
        self.output_dim = output_dim
        #El número de ejmplos
        self.num_examples = num_examples
        # h be the number of hidden units
        h = 128

        #Dimensiones de los vectores-palabra
        dim = 300
        nn_hdim = 100

        #Embedding (este vector se guarda para la siguiente tarea)
        self.C = np.random.randn(dim, input_dim) / np.sqrt(input_dim)

        #Capa oculta
        #U (a |V | × h matrix) - hidden-to-output weights
        self.U = np.random.randn(nn_hdim, dim) / np.sqrt(dim)
        self.b = np.zeros((1, nn_hdim)) #bias

        #Capa de salida
        # W (a |V | × (n − 1)m matrix) word features to output weights
        self.W = np.random.randn(input_dim-1, nn_hdim) / np.sqrt(nn_hdim)
        self.c = np.zeros((1, input_dim-1))
    
    def train(self, bigrams):
        its = 10
        eta = 0.1
        for i in range(0,its):
            print('train {}'.format(i))
            for ex in bigrams:
                #Forward
                #Embedimiento
                c_w = self.C.T[ex[0]]
                #capa oculta
                h1 = np.tanh(np.dot(self.U,c_w) + self.b)[0]
                #salida
                out = np.exp(np.dot(self.W,h1) + self.c)[0]
                #Softmax
                f = out/out.sum(0)

                #Backprop
                #Variable de salida
                d_out = f
                d_out[ex[1]] -= 1
                
                #Variable para la capa oculta
                d_tanh = (1-h1**2)*np.dot(self.W.T,d_out)
                
                #Variable de embedding
                d_emb = np.dot(self.U.T, d_tanh)

                #Actualizacion de salida
                self.W -= eta*np.outer(d_out,h1)
                #Actualiza bias de salida
                self.c -= eta*d_out #[j]

                #Actualizacion de capa oculta
                self.U -= eta*np.outer(d_tanh,c_w)
                #Actualiza bias
                self.b -= eta*d_tanh

                #Actualizacion de embedding
                self.C.T[ex[0]] -= eta*d_emb

    def forward(self, x):    
        #Embedimiento
        x_w = self.C.T[x] #x(k) ← C(wt−k)
        #a ← tanh(Hx + d)
        a = np.tanh(np.dot(self.U, x_w) + self.b)[0]
        # p_j ← e**(a.U + b_j) 
        # if (direct connections) e**(e**(a.U + b_j) + x.W_j)
        out = np.exp(np.dot(self.W, a) + self.c)[0]
        # Normalize the probabilities
        self.p = out/out.sum(0)
        return self.p

    def plot_words(self, ids):
        Z = self.C.T[:-2]
        Z = PCA(2).fit_transform(Z)
        r=0
        plt.scatter(Z[:,0],Z[:,1], marker='o', c='blue')
        for label,x,y in zip(ids, Z[:,0], Z[:,1]):
            plt.annotate(label, xy=(x,y), xytext=(-1,1), 
                         textcoords='offset points', 
                         ha='center', va='bottom')
            r+=1
        plt.show()

    def get_entropy(self, test_data):
        #Inicialización Entropía
        H = 0.0
        Hs = []

        #Evaluamos en el corpus de evaluación
        for sentence in test_data:
            # Número de palabras
            M = len(sentence)
            H_sentence = 0.0
            for cad in sentence:
                # Probabilidad de la cadena
                H_sentence += np.log2(self.forward(word_to_index[cad]))
                
            Hs.append(H_sentence * 1/(-M+1e-100))
            #Obtenemos la entropía cruzada de la cadena
            H += H_sentence * 1/(-M+1e-100)

        return H/len(test_data), Hs
    
    def test(self, test):
        entropy, entropies = self.get_entropy2(test)
        #Visualización del modelo
        print("Promedio de entropias: ",entropy)
        plt.scatter(entropies, entropies, color='g', s=4, label='Bigram model')
        plt.legend(bbox_to_anchor=(1, 1))
        plt.show()

## Pruebas conjunto de Validación

In [ ]:
bengio = Bengio(len(word_to_indexV) + 1, len(word_to_indexV) + 1, len(bigramsV))
bengio.train(bigramsV)

In [ ]:
label = [w[0] for w in sorted(list(word_to_indexV.items())[:5], key=itemgetter(1))]
bengio.plot_words(label)

Calculamos la entropia

In [ ]:
bengio.test2(test)

## Pruebas conjunto de Entrenamiento

In [ ]:
bengio = Bengio(len(word_to_index) + 1, len(word_to_index) + 1, len(bigrams))
bengio.train(bigrams)
label = [w[0] for w in sorted(list(word_to_index.items()), key=itemgetter(1))]
bengio.plot_words(label)

Calculamos la entropia

In [ ]:
bengio.test2(test)